In [ ]:
#두 개 합친거!

from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys 
import pandas as pd
import os
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome()

# 검색 후 페이지 접속하기
trip_list = ['제주김녕미로공원', '주상절리대']
url = "https://www.visitjeju.net/kr/search?q="
time.sleep(1)

# 페이지를 넘겨보자.
CSS_btn_next = "#paging > a.spr_com.page-next"
def go_next_page():
    next_btn = driver.find_element_by_css_selector(CSS_btn_next)
    driver.find_element_by_class_name('spr_com.page-next').send_keys(Keys.ENTER)

for search in range(0, len(trip_list)):
    driver.get(url+trip_list[search])
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, '#content > div > div.cont_wrap > div > div.result_box_r > div.sort_wrap > ul > li:nth-child(3) > a').click()
    time.sleep(1)

    title="//*[@id='content']/div/div[2]/div/div[2]/div[2]/div[1]/div/ul/li[1]/a/p[1]"
    driver.find_element(By.XPATH, title).click()
    time.sleep(3)
    
    driver.switch_to.window(driver.window_handles[1])
    elem = driver.find_element_by_xpath('//*[@id="tab3"]/h2/a')
    actions = ActionChains(driver)
    actions.move_to_element(elem).perform()
    
    CSS_review="#tab3 > h2 > a"
    driver.find_element(By.CSS_SELECTOR, CSS_review).click()
    title = driver.find_element_by_css_selector("#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3")

    result_df = pd.DataFrame()  
    page = 0

    while True:
         
        try:
            while page < 5:
            
                review = driver.find_element_by_css_selector(f'#contentsReviewItem{page} > div > div.user_content > div.review.clear > p.review_txt')
                df = pd.DataFrame({'content': review.text}, index = [0])
                result_df = pd.concat([result_df, df], ignore_index=True)
                page += 1
                time.sleep(1)
        
        except NoSuchElementException:
            break

        page = 0
        go_next_page()
        
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
    if search < len(trip_list):
        if(os.path.exists('visitjeju.csv')):
            result_df.to_csv('visitjeju.csv',mode='a', encoding='utf-8-sig', header=False)
        else:
            result_df.to_csv('visitjeju.csv',mode='w', encoding='utf-8-sig')